In [ ]:
!pip install datasets

from datasets import load_dataset
import re
import pandas as pd
import unicodedata
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
import json
import numpy as np

In [ ]:
#1 — стримим датасет
# ссылка: https://huggingface.co/datasets/missvector/multi-wiki-grammar
def streaming():
    dataset = load_dataset("missvector/multi-wiki-grammar")
    df = dataset["train"].to_pandas()
     # Обрезаем до первых 1000 строк
    df = df.head(1000)
    return df

In [ ]:
# Загружаем для просмотра
df = streaming()
df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

train-00000-of-00011.parquet:   0%|          | 0.00/446M [00:00<?, ?B/s]

train-00001-of-00011.parquet:   0%|          | 0.00/241M [00:00<?, ?B/s]

train-00002-of-00011.parquet:   0%|          | 0.00/176M [00:00<?, ?B/s]

...

train-00008-of-00011.parquet:   0%|          | 0.00/171M [00:00<?, ?B/s]

train-00009-of-00011.parquet:   0%|          | 0.00/183M [00:00<?, ?B/s]

train-00010-of-00011.parquet:   0%|          | 0.00/161M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/356534 [00:00<?, ? examples/s]

,Language,Title,Text,Sentences,Grammatical Sentences,Non-Grammatical Sentences
0,English,School,Implementation of these practices varies.,[Implementation of these practices varies.],[Implementation of these practices varies.],"[{'Error Type': 'Word Order Errors', 'ID': 0, ..."
1,Russian,Sample Sentence,Кот сидит на ковре.,[Кот сидит на ковре.],[Кот сидит на ковре.],"[{'Error Type': 'Agreement Error', 'ID': 0, 'S..."
2,Chinese,Sample Sentence,猫坐在垫子上。,[猫坐在垫子上。],[猫坐在垫子上。],"[{'Error Type': 'Word Order Errors', 'ID': 0, ..."
3,English,Sample Sentence,The cat sits on the mat.,[The cat sits on the mat.],[The cat sits on the mat.],"[{'Error Type': 'Agreement Error', 'ID': 0, 'S..."
4,Russian,Литва,"Литва. На севере граничит с Латвией, на юго-в...","[Литва., На севере граничит с Латвией, на юго...",[По площади и населению является самым крупным...,"[{'Error Type': 'N/A', 'ID': 0, 'Sentence': ''..."


In [ ]:
df.tail()

,Language,Title,Text,Sentences,Grammatical Sentences,Non-Grammatical Sentences
995,Russian,Оман,"Оман — государство в Передней Азии, на юго-во...","[Оман — государство в Передней Азии, на юго-в...","[Оман — государство в Передней Азии, на юго-в...","[{'Error Type': 'N/A', 'ID': 0, 'Sentence': ''..."
996,Russian,Питкэрн,Острова Питкэрн — единственная заморская терри...,[Острова Питкэрн — единственная заморская терр...,[Острова Питкэрн — единственная заморская терр...,"[{'Error Type': 'N/A', 'ID': 0, 'Sentence': ''..."
997,Russian,Румыния,"Румыния — государство в Юго-Восточной Европе,...",[Румыния — государство в Юго-Восточной Европе...,"[Площадь территории — 238 391 км2, население —...","[{'Error Type': 'N/A', 'ID': 0, 'Sentence': ''..."
998,Russian,Сан-Марино,Сан-Марино — одно из самых маленьких государст...,[Сан-Марино — одно из самых маленьких государс...,[Сан-Марино — одно из самых маленьких государс...,"[{'Error Type': 'N/A', 'ID': 0, 'Sentence': ''..."
999,Russian,Сьерра-Леоне,"Сьерра-Леоне — государство в Западной Африке,...",[Сьерра-Леоне — государство в Западной Африке...,[Сьерра-Леоне — государство в Западной Африке...,"[{'Error Type': 'N/A', 'ID': 0, 'Sentence': ''..."


In [ ]:
# Здесь есть проблема: Й заменяется на И
# чистим, оставляем только слова на русском
# [a-яА-Я]
#def cleaning(text):
#    if pd.isna(text):
#        return text
    # Одним regex удаляем всё, кроме русских букв и пробелов + чистим пробелы
#    cleaned_text = re.sub(r'[^а-яА-ЯёЁ\s]+|\s+', ' ', str(text)).strip()
#    return cleaned_text if cleaned_text else pd.NA

In [ ]:
#2 — чистим датасет
# Пробуем нормализацию и проверку на тип данных, чтобы не терять Й
def cleaning(text):
    if pd.isna(text):
        return text
    if not isinstance(text, str):
        return text
    # Нормализуем в NFC (каноническая форма)
    text = unicodedata.normalize("NFC", text)
    cleaned_text = re.sub(r'[^а-яА-ЯёЁ\s]+|\s+', ' ', text).strip()
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text if cleaned_text else pd.NA

In [ ]:
df['Grammatical Sentences'].apply(lambda x: [cleaning(string) for string in x])

,Grammatical Sentences
0,[Implementation of these practices varies.]
1,[Кот сидит на ковре.]
2,[猫坐在垫子上。]
3,[The cat sits on the mat.]
4,[По площади и населению является самым крупным...
...,...
995,"[Оман — государство в Передней Азии, на юго-во..."
996,[Острова Питкэрн — единственная заморская терр...
997,"[Площадь территории — 238 391 км2, население —..."
998,[Сан-Марино — одно из самых маленьких государс...


In [ ]:
df['Grammatical Sentences'].dropna().to_list

,Grammatical Sentences
0,[Implementation of these practices varies.]
1,[Кот сидит на ковре.]
2,[猫坐在垫子上。]
3,[The cat sits on the mat.]
4,[По площади и населению является самым крупным...
...,...
995,"[Оман — государство в Передней Азии, на юго-во..."
996,[Острова Питкэрн — единственная заморская терр...
997,"[Площадь территории — 238 391 км2, население —..."
998,[Сан-Марино — одно из самых маленьких государс...


In [ ]:
# 3 — создаем список сникальных слов и дублируем его (основа для разделения на слоги)

# Функция для получения уникальных слов
def get_unique_words(sents):
    # Объединяем все строки столбца в один текст
    all_text = ' '.join()

    # Преобразуем текст в нижний регистр и токенизируем текст
    tokens = word_tokenize(all_text.lower())

    # Сохраняем уникальные слова
    unique_words = set(tokens)
    unique_words_array = np.array(list(unique_words))
    return unique_words_array

In [ ]:
# Получаем уникальные слова для всего столбца
unique_words = get_unique_words(df)

# Добавляем массив NumPy как новый столбец в DataFrame
# Поскольку список уникальных слов может быть длинным, подгоняем его под размер DataFrame
df_unique_words = pd.DataFrame(list(unique_words), columns=['Words'])

df_unique_words

TypeError: sequence item 0: expected str instance, list found

In [ ]:
# Создаем второй столбец как копию первого
df_unique_words["Hyphenated_Words"] = df_unique_words["Words"]
# теперь ко второму столбцу надо применить токенизацию по слогам

In [ ]:
#3 — разделение на слоги
def syllables(text):
    vowel = 'аеёиоуыэюя'
    cons = 'бвгджзлмнрсхцчшщ'
    brief = 'кпстф'
    voiced = 'й'
    deaf = 'ьъ'
    other = 'бвгджзйклмнпрсстфхцчшщ'

    def _is_not_last_sep(txt):
        return any(ch in vowel for ch in txt)

    def _add_sep(force_sep):
        nonlocal current_syllable, syllables
        if force_sep:
            current_syllable += ' '
            return

        if not re.search(f'[{vowel}]', current_syllable):
            return

        syllables.append(current_syllable)
        current_syllable = ''

    syllables = []
    current_syllable = ''
    words = text.split()

    for word in words:
        if len(word) < 2:
            word = word.replace('-', '')

        for i, char in enumerate(word):
            current_syllable += char
            next_char = word[i+1] if i+1 < len(word) else ''

            if not next_char or next_char not in 'абвгдеёжзийклмнопрстуфхцчшщыьэюя':
                continue

            if (i != 0 and i != len(word)-1 and char in voiced and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i < len(word)-1 and char in vowel and
                next_char in vowel):
                _add_sep(False)
                continue

            if (i < len(word)-2 and char in vowel and
                word[i+1] in other and word[i+2] in vowel):
                _add_sep(False)
                continue

            if (i < len(word)-2 and char in vowel and
                word[i+1] in brief and word[i+2] in other and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i > 0 and i < len(word)-1 and char in cons and
                word[i-1] in vowel and next_char not in vowel and
                next_char not in deaf and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i < len(word)-1 and char in deaf and
                (next_char not in vowel or _is_not_last_sep(word[:i]))):
                _add_sep(False)
                continue

        _add_sep(True)

    if current_syllable:
        syllables.append(current_syllable)

    return '-'.join(syllables)

In [ ]:
# Применяем функцию ко всему столбцу "Words" в df_unique_words
df_unique_words['Hyphenated_Words'] = df_unique_words['Words'].apply(syllables)

# Результат
df_unique_words['Hyphenated_Words']

,Hyphenated_Words
0,у-ка-зан-но-е
1,кор-рум-пи-ро-ван-ны-е
2,спе-кта-кля
3,при-нё-сши-ми
4,ле-стных
...,...
185522,за-пре-тным
185523,э-ме-ки
185524,со-ве-тов
185525,тех-но-кра-ты


In [ ]:
 #5 – создаем словарь,  где ключи - слова из первого столбца, а значения - из второго, и сохраняем в формате json
def save(df_unique_words):
    words_dict = dict(zip(df_unique_words["Hyphenated_Words"], df_unique_words["Words"]))

    with open("hyphenated_words_dict.json", "w", encoding="utf-8") as json_file:
        json.dump(words_dict, json_file, ensure_ascii=False, indent=4)

    return words_dict

In [ ]:
# Проверка результата
save(df_unique_words)

{'у-ка-зан-но-е ': np.str_('указанное'),
 'кор-рум-пи-ро-ван-ны-е ': np.str_('коррумпированные'),
 'спе-кта-кля ': np.str_('спектакля'),
 'при-нё-сши-ми ': np.str_('принёсшими'),
 'ле-стных ': np.str_('лестных'),
 'э-фи-рам ': np.str_('эфирам'),
 'китс ': np.str_('китс'),
 'иль-вес ': np.str_('ильвес'),
 'са-мо-за-рож-де-ни-е ': np.str_('самозарождение'),
 'бо-ров-ско-го ': np.str_('боровского'),
 'у-ста-нав-ли-ва-я ': np.str_('устанавливая'),
 'мо-ду-лю ': np.str_('модулю'),
 'по-ю-щи-е ': np.str_('поющие'),
 'ав-стро-а-зи-а-тской ': np.str_('австроазиатской'),
 'и-мел ': np.str_('имел'),
 'гра-би-тель ': np.str_('грабитель'),
 'френ-сис ': np.str_('френсис'),
 'со-бач-ки ': np.str_('собачки'),
 'из-бра-ни-е ': np.str_('избрание'),
 'э-лей-ский ': np.str_('элейский'),
 'душ-но-е ': np.str_('душное'),
 'со-ве-тской ': np.str_('советской'),
 'э-рех-те-йо-на ': np.str_('эрехтейона'),
 'ра-туш-ной ': np.str_('ратушной'),
 'гнчак ': np.str_('гнчак'),
 'двух-це-по-чеч-ной ': np.str_('двухце

**Делим на слоги датасет из Викисловаря**

In [ ]:
!wget data.txt https://raw.githubusercontent.com/vifirsanova/stat-llm/refs/heads/main/morphs_output.txt

with open("morphs_output.txt", "r", encoding="utf-8") as morphs:
    words = morphs.read().strip().split('\n')

--2025-04-24 14:02:12--  http://data.txt/
Resolving data.txt (data.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘data.txt’
--2025-04-24 14:02:12--  https://raw.githubusercontent.com/vifirsanova/stat-llm/refs/heads/main/morphs_output.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9328665 (8.9M) [text/plain]
Saving to: ‘morphs_output.txt’

morphs_output.txt   100%[===================>]   8.90M  --.-KB/s    in 0.09s   

2025-04-24 14:02:13 (94.9 MB/s) - ‘morphs_output.txt’ saved [9328665/9328665]

FINISHED --2025-04-24 14:02:13--
Total wall clock time: 0.5s
Downloaded: 1 files, 8.9M in 0.09s (94.9 MB/s)


In [ ]:
words

['у̀льтравысо̀кочастотными',
 'у̀льтравысо̀кочастотному',
 'у̀льтравысо̀кочастотного',
 'това̀рораспорядительными',
 'това̀рораспорядительному',
 'това̀рораспорядительного',
 'самосовершенствовавшийся',
 'са̀мопрограммировавшийся',
 'рефлексотерапевтическому',
 'рефлексотерапевтического',
 'рефлексотерапевтическими',
 'профессионализировавшись',
 'пирамидальнометельчатыми',
 'пирамидальнометельчатому',
 'пирамидальнометельчатого',
 'недоукомплектовывающийся',
 'недоукомплектовывавшийся',
 'наэкспериментировавшийся',
 'меланоцѝтостимулирующими',
 'меланоцѝтостимулирующему',
 'меланоцѝтостимулирующего',
 'кремнефтористоводородных',
 'кремнефтористоводородным',
 'кремнефтористоводородный',
 'кремнефтористоводородные',
 'кремнефтористоводородную',
 'кремнефтористоводородною',
 'кремнефтористоводородном',
 'кремнефтористоводородной',
 'кремнефтористоводородное',
 'кремнефтористоводородная',
 'интернационалистическому',
 'интернационалистического',
 'интернационалистическими',
 'институциона

In [ ]:
import pandas as pd
word_series = pd.Series(words, name = 'words')
word_series

,words
0,у̀льтравысо̀кочастотными
1,у̀льтравысо̀кочастотному
2,у̀льтравысо̀кочастотного
3,това̀рораспорядительными
4,това̀рораспорядительному
...,...
400359,злю
400360,ищу
400361,эти
400362,ссу


In [ ]:
# Создаём DataFrame с двумя одинаковыми столбцами
df = pd.DataFrame({
    'words': word_series,
    'hyphenated_words': word_series  # дубликат
})
df

,words,hyphenated_words
0,у̀льтравысо̀кочастотными,у̀льтравысо̀кочастотными
1,у̀льтравысо̀кочастотному,у̀льтравысо̀кочастотному
2,у̀льтравысо̀кочастотного,у̀льтравысо̀кочастотного
3,това̀рораспорядительными,това̀рораспорядительными
4,това̀рораспорядительному,това̀рораспорядительному
...,...,...
400359,злю,злю
400360,ищу,ищу
400361,эти,эти
400362,ссу,ссу


In [ ]:
# Разделение на слоги
import re
def syllables(text):
    vowel = 'аеёиоуыэюя'
    cons = 'бвгджзлмнрсхцчшщ'
    brief = 'кпстф'
    voiced = 'й'
    deaf = 'ьъ'
    other = 'бвгджзйклмнпрсстфхцчшщ'

    def _is_not_last_sep(txt):
        return any(ch in vowel for ch in txt)

    def _add_sep(force_sep):
        nonlocal current_syllable, syllables
        if force_sep:
            current_syllable += ' '
            return

        if not re.search(f'[{vowel}]', current_syllable):
            return

        syllables.append(current_syllable)
        current_syllable = ''

    syllables = []
    current_syllable = ''
    words = text.split()

    for word in words:
        if len(word) < 2:
            word = word.replace('-', '')

        for i, char in enumerate(word):
            current_syllable += char
            next_char = word[i+1] if i+1 < len(word) else ''

            if not next_char or next_char not in 'абвгдеёжзийклмнопрстуфхцчшщыьэюя':
                continue

            if (i != 0 and i != len(word)-1 and char in voiced and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i < len(word)-1 and char in vowel and
                next_char in vowel):
                _add_sep(False)
                continue

            if (i < len(word)-2 and char in vowel and
                word[i+1] in other and word[i+2] in vowel):
                _add_sep(False)
                continue

            if (i < len(word)-2 and char in vowel and
                word[i+1] in brief and word[i+2] in other and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i > 0 and i < len(word)-1 and char in cons and
                word[i-1] in vowel and next_char not in vowel and
                next_char not in deaf and
                _is_not_last_sep(word[i+1:])):
                _add_sep(False)
                continue

            if (i < len(word)-1 and char in deaf and
                (next_char not in vowel or _is_not_last_sep(word[:i]))):
                _add_sep(False)
                continue

        _add_sep(True)

    if current_syllable:
        syllables.append(current_syllable)

    return '-'.join(syllables)

In [ ]:
df['hyphenated_words'] = df['hyphenated_words'].apply(lambda x: syllables(x))
df

,words,hyphenated_words
0,у̀льтравысо̀кочастотными,у̀ль-тра-вы-со̀ко-ча-сто-тны-ми
1,у̀льтравысо̀кочастотному,у̀ль-тра-вы-со̀ко-ча-сто-тно-му
2,у̀льтравысо̀кочастотного,у̀ль-тра-вы-со̀ко-ча-сто-тно-го
3,това̀рораспорядительными,то-ва̀ро-ра-спо-ря-ди-тель-ны-ми
4,това̀рораспорядительному,то-ва̀ро-ра-спо-ря-ди-тель-но-му
...,...,...
400359,злю,злю
400360,ищу,и-щу
400361,эти,э-ти
400362,ссу,ссу


In [ ]:
df.to_csv('/content/syllables.csv', index=False, encoding='utf-8')